In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
kaynak_klasoru = 'animals/animals' # https://www.kaggle.com/datasets/ashishsaxena2209/animal-image-datasetdog-cat-and-panda
photo_size = 64
resim_boyutu = (photo_size, photo_size)
batch_size = 64

# Veri yükleyicisini oluşturmak için ImageDataGenerator sınıfını kullanabilirsiniz:



In [3]:
veri_yukleyici = ImageDataGenerator(rescale=1./255)
veri_seti = veri_yukleyici.flow_from_directory(
    kaynak_klasoru,
    target_size=resim_boyutu,
    batch_size=batch_size,
    class_mode='categorical')

Found 3000 images belonging to 3 classes.


# Modeli oluşturmak için Sequential sınıfını kullanabilirsiniz:


In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(photo_size, photo_size, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Eğitim

In [5]:
epochs = 10
model.fit(veri_seti, epochs=epochs)

Epoch 1/10
47/47 [==============================] - 13s 254ms/step - loss: 0.8619 - accuracy: 0.5283
Epoch 2/10
47/47 [==============================] - 13s 281ms/step - loss: 0.7212 - accuracy: 0.6270
Epoch 3/10
47/47 [==============================] - 14s 286ms/step - loss: 0.6204 - accuracy: 0.6860
Epoch 4/10
47/47 [==============================] - 13s 277ms/step - loss: 0.6039 - accuracy: 0.6957
Epoch 5/10
47/47 [==============================] - 14s 292ms/step - loss: 0.5702 - accuracy: 0.7177
Epoch 6/10
47/47 [==============================] - 13s 270ms/step - loss: 0.5275 - accuracy: 0.7477
Epoch 7/10
47/47 [==============================] - 12s 262ms/step - loss: 0.4708 - accuracy: 0.7793
Epoch 8/10
47/47 [==============================] - 13s 273ms/step - loss: 0.4147 - accuracy: 0.8113
Epoch 9/10
47/47 [==============================] - 13s 274ms/step - loss: 0.3694 - accuracy: 0.8403
Epoch 10/10
47/47 [==============================] - 13s 272ms/step - loss: 0.3330 - accura

## modeli model.h5 isiyle kaydedelim

In [6]:
model.save("model1.h5")

# TestKodu
* 1-kutuphane Ekleme
* 2- modelin yüklenmesi
* 3-test verisi okunması
* 4- resimleri  tek tek istenilen formata dönüştürüp model ile tahminin yapılması

In [7]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

# Eğitilmiş modelinizi yükleyin
model = load_model( 'model.h5')

In [9]:
def TrainModel(epochs = 10,photo_size = 64,batch_size = 64):
    kaynak_klasoru = 'animal' # https://www.kaggle.com/datasets/ashishsaxena2209/animal-image-datasetdog-cat-and-panda
    resim_boyutu = (photo_size, photo_size)
    
    veri_yukleyici = ImageDataGenerator(rescale=1./255)
    veri_seti = veri_yukleyici.flow_from_directory(
        kaynak_klasoru,
        target_size=resim_boyutu,
        batch_size=batch_size,
        class_mode='categorical')


    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(photo_size, photo_size, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    
    model.fit(veri_seti, epochs=epochs)
    return model

In [10]:
def TestModel(model,show=False,folderName="dogs"):
    
    test_folder = f'test_images/{folderName}'  # Test verilerinizin bulunduğu klasör yolu

    ppp=[]

    # Test resimlerinizi yükleyin ve tahmin yapın
    for filename in os.listdir(test_folder):
        img_path = os.path.join(test_folder, filename)
        img = Image.open(img_path)
        img = img.resize((photo_size, photo_size))  # Resmi 64x64 boyutuna yeniden boyutlandırın
        img = np.array(img) / 255.0  # Resimleri [0, 1] aralığında ölçeklendirin
        img = np.expand_dims(img, axis=0)  # Girdi tensörü boyutunu (1, 64, 64, 3) yapın

        # Tahmin yapın
        pred = model.predict(img)
        prediction = np.argmax(pred, axis=1)

        # Tahmin sonucunu yazdırın
        if show:
            print(f"{filename}: {prediction}") 

        # yazdırmak yerine sonucu listeye ekleyip değerlendirlim
        ppp.append(prediction)
        
        

    return ppp

In [11]:
a=TestModel(model,True,"dogs")
print(f"tahim sonucu \n  kedi : {a.count(0)} \n  kopek : {a.count(1)} \n  panda : {a.count(2)} \n  toplam tahmin{len(a)}")
print("doğruluk",a.count(1) /len(a))


1/1 [==============================] - 0s 18ms/step
dog.jpg: [1]
1/1 [==============================] - 0s 17ms/step
dogs_00001.jpg: [1]
1/1 [==============================] - 0s 18ms/step
dogs_00002.jpg: [1]
1/1 [==============================] - 0s 17ms/step
dogs_00003.jpg: [0]
1/1 [==============================] - 0s 18ms/step
dogs_00004.jpg: [0]
1/1 [==============================] - 0s 20ms/step
dogs_00005.jpg: [1]
1/1 [==============================] - 0s 17ms/step
dogs_00006.jpg: [1]
1/1 [==============================] - 0s 17ms/step
dogs_00007.jpg: [0]
1/1 [==============================] - 0s 16ms/step
dogs_00008.jpg: [1]
1/1 [==============================] - 0s 17ms/step
dogs_00009.jpg: [1]
1/1 [==============================] - 0s 18ms/step
dogs_00010.jpg: [0]
1/1 [==============================] - 0s 16ms/step
dogs_00011.jpg: [1]
1/1 [==============================] - 0s 16ms/step
dogs_00012.jpg: [1]
1/1 [==============================] - 0s 18ms/step
dogs_00013.jpg: [1]

In [12]:
a=TestModel(model,False,"cats")
print(f"tahim sonucu \n  kedi : {a.count(0)} \n  kopek : {a.count(1)} \n  panda : {a.count(2)} \n  toplam tahmin{len(a)}")
print("doğruluk", (a.count(0)/len(a)))


1/1 [==============================] - 0s 18ms/step
tahim sonucu 
  kedi : 12 
  kopek : 2 
  panda : 0 
  toplam tahmin14
doğruluk 0.8571428571428571


In [73]:
#myModel=TrainModel()
#a=TestModel(myModel,False,"dogs")

Found 1958 images belonging to 2 classes.
Epoch 1/10
31/31 [==============================] - 10s 304ms/step - loss: 0.6963 - accuracy: 0.5026
Epoch 2/10
31/31 [==============================] - 9s 291ms/step - loss: 0.6856 - accuracy: 0.5751
Epoch 3/10
31/31 [==============================] - 9s 294ms/step - loss: 0.6618 - accuracy: 0.5858
Epoch 4/10
31/31 [==============================] - 9s 278ms/step - loss: 0.6359 - accuracy: 0.6272
Epoch 5/10
31/31 [==============================] - 9s 284ms/step - loss: 0.6019 - accuracy: 0.6691
Epoch 6/10
31/31 [==============================] - 9s 283ms/step - loss: 0.5707 - accuracy: 0.6859
Epoch 7/10
31/31 [==============================] - 9s 290ms/step - loss: 0.5282 - accuracy: 0.7416
Epoch 8/10
31/31 [==============================] - 11s 363ms/step - loss: 0.4917 - accuracy: 0.7584
Epoch 9/10
31/31 [==============================] - 11s 355ms/step - loss: 0.4731 - accuracy: 0.7712
Epoch 10/10
31/31 [==============================] - 9s